<a id="top" style="float:right;" href="http://dynfluid.ensam.eu/"><img style="height:120px;" src="http://dynfluid.ensam.eu/uas/DYNFLUID/logoPrincipal/Logo-DynFluid-Web.png"/></a>
<a style="float:left;" href="http://www.cnam.fr//"><img style="height:120px;" src="https://upload.wikimedia.org/wikipedia/commons/4/45/Logo_ENSTA_Paris.jpg"></a>

<center>
<h3 style="color:#888888;"> <i>--  Introduction à la méthode de Boltzmann sur Réseau  --</i> </h3>
<h1> TP n°4 </h1>
<h3> Modèles de collision Avancés </h3>

<h6><a href="mailto:simon.marie@lecnam.net">simon.marie@lecnam.net</a></h6>
</center>

In [64]:
import numpy as np
import matplotlib.pyplot as plt
#from numba import jit
import time
fs=20
plt.style.use('ggplot')
plt.rc('xtick',labelsize=fs)
plt.rc('ytick',labelsize=fs)
%matplotlib inline

<h1> Préambule </h1>

Le TP doit être rendu sous la forme d'un Notebook jupyter en respectant la nomenclature suivante:
<center>
<b>TP4_NOM1_NOM2.ipynb</b>
</center>

Tous les résultats, discussions, analyses, doivent donc être inclus dans le fichier.

<h1>Présentation du TP</h1>

On se propose dans ce TP de reprendre les simulations réalisées dans les TP2 et 3 et d'appliquer un modèle de collision avancé de type MRT ou régularisation (au choix). On utilisera dans ce TP le réseau de vitesse D2Q9.


Vous pouvez donc reprendre soit le TP2: *Ecoulement dans une cavité entrainée*, soit le TP3: *Ecoulement autour d'un cylindre* en reprenant également les mêmes conditions aux limites mais en remplacant la collision par un des deux modèles avancés présentés ci-dessous.


In [65]:
ca=np.array([[0,0],[1,0],[0,1],[-1,0],[0,-1],[1,1],[-1,1],[-1,-1],[1,-1]])
w=[4./9.,1./9.,1./9.,1./9.,1./9.,1./36.,1./36.,1./36.,1./36.]
c0=1/np.sqrt(3)

<h2>Le modèle à temps de relaxation multiple MRT</h2>

Le principe du modèle MRT est de réaliser l'étape de collision dans l'espace des moments. Pour cela il faut donc définir 9 moments indépendants. La construction de ces moments fait l'objet de nombreux articles et ne sera pas détaillé ici. On donne ici la version classique qui utilise la matrice de passage suivante pour passer des distributions aux moments: 

$$
\mathbf{M}=\left(
\begin{array}{ccccccccc}
1&1&1&1&1&1&1&1&1\\
-4&-1&-1&-1&-1&2&2&2&2\\
4&-2&-2&-2&-2&1&1&1&1\\
0&1&0&-1&0&1&-1&-1&1\\
0&-2&0&2&0&1&-1&-1&1\\
0&0&1&0&-1&1&1&-1&-1\\
0&0&-2&0&2&1&1&-1&-1\\
0&1&-1&1&-1&0&0&0&0\\
0&0&0&0&0&1&-1&1&-1\\
\end{array}
\right)
$$

Les moments sont donc obtenus à partir des distributions:

$$
\mathbf{m}=\mathbf{M}\mathbf{g}
$$

et la partie équilibre:

$$
\mathbf{m}^{eq}=\mathbf{M}\mathbf{g}^{eq}
$$

La collision devient alors:

$$
\displaystyle{g_{\alpha}^{coll} = g_{\alpha} - \mathbf{M}^{-1}\mathbf{S}[\mathbf{m}-\mathbf{m}^{eq}] }
$$

Où $\mathbf{S}$ est la matrice des temps de relaxation qui peuvent être fixés indépendemment pour chacun des moments. La littérature base le choix de ses temps de relaxation sur une optimisation de la stabilité et donne:

$$
\displaystyle{S=\text{diag}(0,1.05,1.1,0,1.25,0,1.25,1/\tau_g,1/\tau_g)}
$$


In [66]:
M=np.array([[1,1,1,1,1,1,1,1,1],[-4,-1,-1,-1,-1,2,2,2,2],[4,-2,-2,-2,-2,1,1,1,1],
           [0,1,0,-1,0,1,-1,-1,1],[0,-2,0,2,0,1,-1,-1,1],[0,0,1,0,-1,1,1,-1,-1],
           [0,0,-2,0,2,1,1,-1,-1],[0,1,-1,1,-1,0,0,0,0],[0,0,0,0,0,1,-1,1,-1]])

In [67]:

# Maillage/Domaine
R=5        # rayon du cylindre
x0,y0=8*R,10*R # centre du cylindre
nx=40*R
ny=20*R

# Taille de maille:
dx=1/(nx)

# Vitesse du son réelle à  293K:
c0_real=330

# Pas de temps:
dt=dx/c0_real*c0

# Mach:
M0=0.3
U0 = M0*c0
nu = U0*(2*R)/Re #La longueur du domaine est unitaire
taug=1/2 + nu/(c0**2)
S=np.diag([0,1.05,1.1,0,1.25,0,1.25,1/taug,1/taug])
N = np.linalg.inv(M)

In [68]:
def init(M0,Re):
    # Initialisation du domaine
    rho = np.ones((ny,nx))
    U0 = c0*M0
    ux = U0*np.ones((ny,nx))
    uy = np.zeros((ny,nx))
    nu = U0*(2*R)/Re #La longueur du domaine est unitaire
    taug = 1/2 + nu/(c0**2)
    geq = np.zeros((ny,nx,9))
    for i in range(9):
        geq[:,:,i] = w[i]*(1 + ca[i][0]*M0*c0 + 9/2*(ca[i][0]*M0*c0)**2 - 3/2*(M0*c0)**2)
    return geq,rho,ux,uy,taug,U0 

In [79]:
def collide(gcoll,g,M,S):
    m = np.zeros(9)
    meq = np.zeros(9)
    m = np.einsum('ij,jz->iz',np.array(M),np.array(g))
    meq = np.einsum('ij,jz->iz',M,geq)
    
    gcoll[:,:,:]=g[:,:,:] - N*S*(m - meq)

In [70]:
def eq(rho,ux,uy,geq):
    # Mise à  jour de geq
    for i in range(9):
        geq[:,:,i] = w[i] * rho[:,:]*(1 + 3*(ca[i][0]*ux[:,:] + ca[i][1]*uy[:,:]) + 9/2*(ca[i][0]*ux[:,:] + ca[i][1]*uy[:,:])**2 - 3/2*(ux[:,:]**2 + uy[:,:]**2))

In [71]:
def propagate(g,gcoll):
    # Etape de propagation
    g[:,:,0] = gcoll[:,:,0]
    g[:,1:,1] = gcoll[:,:-1,1]
    g[1:,:,2] = gcoll[:-1,:,2]
    g[:,:-1,3] = gcoll[:,1:,3]
    g[:-1,:,4] = gcoll[1:,:,4]
    g[1:,1:,5] = gcoll[:-1,:-1,5]
    g[1:,:-1,6] = gcoll[:-1,1:,6]
    g[:-1,:-1,7] = gcoll[1:,1:,7]
    g[:-1,1:,8] = gcoll[1:,:-1,8]

In [72]:
def macro(g,rho,ux,uy):
    # calcul des variables macro
    rho[:,:] = g[:,:,0] + g[:,:,1] + g[:,:,2] + g[:,:,3] + g[:,:,4] + g[:,:,5] + g[:,:,6] + g[:,:,7] + g[:,:,8]
    ux[:,:] = (g[:,:,1] - g[:,:,3] + g[:,:,5] - g[:,:,6] - g[:,:,7] + g[:,:,8])/rho[:,:]
    uy[:,:] = (g[:,:,2] - g[:,:,4] + g[:,:,5] + g[:,:,6] - g[:,:,7] - g[:,:,8])/rho[:,:]

In [73]:
def wall_slip(gcoll,g,mask):
    # Paroi solide sur le cylindre: Bounce back avec frottement
    g[mask==1,1] = gcoll[mask==1,3]
    g[mask==1,2] = gcoll[mask==1,4]
    g[mask==1,3] = gcoll[mask==1,1]
    g[mask==1,4] = gcoll[mask==1,2]
    g[mask==1,5] = gcoll[mask==1,7]
    g[mask==1,6] = gcoll[mask==1,8]
    g[mask==1,7] = gcoll[mask==1,5]
    g[mask==1,8] = gcoll[mask==1,6]
    

In [74]:
def inflow(g,rho,ux,uy,M0):
    # Conditions de vitesse au bord superieur
    U0 = M0*c0
    ux[:,0]= U0
    # Macro: On impose ux et on en déduit rho_in
    rho[:,0] = 1/(1-U0)*(2*(g[:,0,3] + g[:,0,7] + g[:,0,6]) + g[:,0,2] + g[:,0,4] + g[:,0,0])
    
    # Distributions: On impose les distributions inconnues avec le Bounce-Back hors equilibre 
    g[:,0,1] = g[:,0,3] + 2/3*U0*rho[:,0]
    g[:,0,5] = g[:,0,7] + 1/6*(U0 + uy[:,0])*rho[:,0]
    g[:,0,8] = g[:,0,6] + 1/6*(U0 - uy[:,0])*rho[:,0]

In [75]:
def outflow(g,gcoll):
    # Conditions de gradient nul
    # Out
    g[:,-1,3] = g[:,-2,3]
    g[:,-1,6] = g[:,-2,6]
    g[:,-1,7] = g[:,-2,7]
    # Top
    g[-1,:,7] = g[-2,:,7]
    g[-1,:,4] = g[-2,:,4]
    g[-1,:,8] = g[-2,:,8]
    # Bottom
    g[0,:,2] = g[1,:,2]
    g[0,:,6] = g[1,:,6]
    g[0,:,5] = g[1,:,5]

In [76]:
# Nombre de Reynolds:
Re = 10

# initialisation:
geq,rho,ux,uy,taug,U0=init(M0,Re)
g,gcoll=geq.copy(),geq.copy()

# Marquage des conditions aux limites: 
# Fluide: mask=0
# Solide: mask=1
mask=np.zeros((ny,nx));
mask[y0 - R:y0 + R + 1,x0 - R] = 1
mask[y0 - R : y0 + R + 1,x0 + R] = 1
mask[y0 - R, x0 - R: x0 + R + 1] = 1
mask[y0 + R + 1, x0 - R: x0 + R + 1] = 1


In [80]:
nt=8000
Cd=np.zeros(nt)
start=time.time()
# Nombre de Reynolds:
Re = 80

# initialisation:
geq,rho,ux,uy,taug,U0=init(M0,Re)
g,gcoll=geq.copy(),geq.copy()

for it in np.arange(nt):
    collide(gcoll,g,M,S)
    propagate(g,gcoll)
    wall_slip(gcoll,g,mask)
    inflow(g,rho,ux,uy,M0)
    outflow(g,gcoll)
    macro(g,rho,ux,uy)
    eq(rho,ux,uy,geq)
    Cd[it] = dragcoeff(rho,mask,M0)
tcal=time.time()-start
mlups=nx*ny*1e-6*nt/tcal
print(str(nt)+" itérations en "+str(tcal)+"s: Performances: "+str(mlups)+"  MLUPS")

ValueError: operand has more dimensions than subscripts given in einstein sum, but no '...' ellipsis provided to broadcast the extra dimensions.

<h2>Le modèle régularisé</h2>

Le principe du modèle régularisé est de reconstruire la partie hors équilibre en utilisant une estimation à l'aide du tenseur de cisaillement $S_{ij}$ calculé à l'ordre 2 par différences finies:

$$      
\displaystyle{g_\alpha^{heq}=-\dfrac{\omega_\alpha \rho \tau_g \Delta t}{c_0^2}H_{ij}S_{ij}} 
$$


**Attention** ici il y a bien sommation sur i et j: $H_{ij}S_{ij}=H_{11}S_{11}+H_{12}S_{12}+H_{21}S_{21}+H_{22}S_{22}$.

Avec:

$$
H_{ij}=c_{\alpha,i}c_{\alpha,j}-c_0^2 \delta_{ij}
$$

et 

$$
\displaystyle{S_{ij}=\dfrac{1}{2}\left( \dfrac{\partial u_i}{\partial x_j}+\dfrac{\partial u_j}{\partial x_i} \right)}
$$

En remplacant $g$ par $g^{eq}+g^{heq}$, dans l'étape de collision BGK, la nouvelle collision devient:

$$
\displaystyle{g_{\alpha}^{coll} = g_{\alpha}^{eq}+ \left(1-\dfrac{1}{\tau_g}\right)g_{\alpha}^{heq}}
$$

**Note:** On pourra utiliser la fonction <a href="https://numpy.org/doc/stable/reference/generated/numpy.gradient.html">*gradient*</a> de numpy pour évaluer les composante de $S_{ij}$.

In [ ]:
def Sij(St,ux,uy):
    '''
    Calcul du tenseur de cisaillement
    '''
    duxdx = np.gradient(ux,dx,axis = 0)
    duxdy = np.gradient(ux,dy,axis = 1)
    duydx = np.gradient(uy,dx,axis = 0)
    duydy = np.gradient(uy,dy,axis = 1)
    St[:,:,0] = duxdx
    St[:,:,1] = 1/2*(duxdy + duydx)
    St[:,:,2] = duydy
    St[:,:,3] = 1/2*(duydx + duxdy)

In [ ]:
H2 = np.zeros((2,2,9))
for i in range(9):
        H2[0,0,i] = ca[0,0]*ca[0,0] + ca[1,0]*ca[1,0] + ca[2,0]*ca[2,0] + ca[3,0]*ca[3,0] + ca[4,0]*ca[4,0] + ca[5,0]*ca[5,0] + ca[6,0]*ca[6,0] + ca[7,0]*ca[7,0] + ca[8,0]*ca[8,0] - c0**2
        H2[0,1,i] = ca[0,0]*ca[0,1] + ca[1,0]*ca[1,1] + ca[2,0]*ca[2,1] + ca[3,0]*ca[3,1] + ca[4,0]*ca[4,1] + ca[5,0]*ca[5,1] + ca[6,0]*ca[6,1] + ca[7,0]*ca[7,1] + ca[8,0]*ca[8,1]
        H2[1,0,i] = ca[0,0]*ca[0,1] + ca[1,0]*ca[1,1] + ca[2,0]*ca[2,1] + ca[3,0]*ca[3,1] + ca[4,0]*ca[4,1] + ca[5,0]*ca[5,1] + ca[6,0]*ca[6,1] + ca[7,0]*ca[7,1] + ca[8,0]*ca[8,1]
        H2[1,1,i] = ca[0,1]*ca[0,1] + ca[1,1]*ca[1,1] + ca[2,1]*ca[2,1] + ca[3,1]*ca[3,1] + ca[4,1]*ca[4,1] + ca[5,1]*ca[5,1] + ca[6,1]*ca[6,1] + ca[7,1]*ca[7,1] + ca[8,1]*ca[8,1] - c0**2

In [ ]:
def hors_eq(gheq,rho,St,H2):
    '''
    Mise a jour de la partie hors equilibre
    '''
    gheq[:,:,0] = -w[0]*rho[:,:]*taug*dt/c0**2*(H[1,1,0]*S[1,1,0] + H[1,2,0]*S[1,2,0] + H[2,1,0]*S[2,1,0] + H[2,2,0]*S[2,2,0])
    gheq[:,:,1] = -w[1]*rho[:,:]*taug*dt/c0**2*(H[1,1,1]*S[1,1,1] + H[1,2,1]*S[1,2,1] + H[2,1,1]*S[2,1,1] + H[2,2,1]*S[2,2,1])
    gheq[:,:,2] = -w[2]*rho[:,:]*taug*dt/c0**2*(H[1,1,2]*S[1,1,2] + H[1,2,2]*S[1,2,2] + H[2,1,2]*S[2,1,2] + H[2,2,2]*S[2,2,2])
    gheq[:,:,3] = -w[3]*rho[:,:]*taug*dt/c0**2*(H[1,1,3]*S[1,1,3] + H[1,2,3]*S[1,2,3] + H[2,1,3]*S[2,1,3] + H[2,2,3]*S[2,2,3])
    gheq[:,:,4] = -w[4]*rho[:,:]*taug*dt/c0**2*(H[1,1,4]*S[1,1,4] + H[1,2,4]*S[1,2,4] + H[2,1,4]*S[2,1,4] + H[2,2,4]*S[2,2,4])
    gheq[:,:,5] = -w[5]*rho[:,:]*taug*dt/c0**2*(H[1,1,5]*S[1,1,5] + H[1,2,5]*S[1,2,5] + H[2,1,5]*S[2,1,5] + H[2,2,5]*S[2,2,5])
    gheq[:,:,6] = -w[6]*rho[:,:]*taug*dt/c0**2*(H[1,1,6]*S[1,1,6] + H[1,2,6]*S[1,2,6] + H[2,1,6]*S[2,1,6] + H[2,2,6]*S[2,2,6])
    gheq[:,:,7] = -w[7]*rho[:,:]*taug*dt/c0**2*(H[1,1,7]*S[1,1,7] + H[1,2,7]*S[1,2,7] + H[2,1,7]*S[2,1,7] + H[2,2,7]*S[2,2,7])
    gheq[:,:,8] = -w[8]*rho[:,:]*taug*dt/c0**2*(H[1,1,8]*S[1,1,8] + H[1,2,8]*S[1,2,8] + H[2,1,8]*S[2,1,8] + H[2,2,8]*S[2,2,8])
   

In [ ]:
def init_heq(rho,ux,uy):
    '''
    Initialisation de la partie hors eq
    '''
    St = np.zeros((nx,ny,4))
    Sij(St,ux,uy)
    gheq = np.zeros((nx,ny,9))
    hors_eq(gheq,rho,St)
    return St,gheq

In [1]:
def collision(gcoll,geq,gheq):
    '''
    Mise a jour de la collision régularisé
    '''
    gcoll[:,:,:]=...

<h1>Travail demandé</h1>

En reprenant les mêmes conditions initiales et les mêmes conditions aux limites que dans le TP2 ou 3, implémentez une des deux collisions avancées pour effectuer des calculs pour des plus grand nombre de Reynolds. 

Comparez vos résultats à ceux obtenus pour la collision BGK.

# Conclusion

Présenter ici la synthèse de votre TP en décrivant les points importants et les principaux résultats. 


In [1]:
from IPython.core.display import HTML
style=open('notebooks.css', "r").read()
HTML(style)

<a id="top" style="float:right;" href="http://dynfluid.ensam.eu/"><img style="height:120px;" src="http://dynfluid.ensam.eu/uas/DYNFLUID/logoPrincipal/Logo-DynFluid-Web.png"/></a>
<a style="float:left;" href="http://www.cnam.fr//"><img style="height:120px;" src="https://upload.wikimedia.org/wikipedia/commons/4/45/Logo_ENSTA_Paris.jpg"></a>
<center><a href="#top">Retour en haut de la page</a></center>